In this workspace, you will load in a dataset of gene expression data for classifying the leukemia type of samples. The data has 72 samples and every sample contains 7,129 gene expression values.

You will train a linear SVC to classify these samples. However, since many of the features are not so relevant, you will first perform feature selection to identify the columns that are most closely correlated to the target variable. You'll use K-fold CV to select the number of samples to include in your SVC model.

> Most of the points for this question are assigned automatically. However, a small number of points will additionally be manually assigned for code style: minimizing computation where possible, avoiding unnecessary for loops in favor of vectorized operations, etc.

| Name | Type | Description |
| ---- | ---- | ---- |
|`Xtr`	|2d numpy array	|Training data (features).|
|`Xts`	|2d numpy array	|Test data (features).|
|`ytr`	|1d numpy array	|Training data (target).|
|`yts`	|1d numpy array	|Test data (target).|
|`score_ft`	|2d numpy array	|2d numpy array with score of each feature in each fold|
|`score_val`	|2d numpy array	|2d numpy array with validation score (accuracy) of each model in each fold|
|`best_d`	|integer	|The optimal number of features to include (best average validation accuracy).|
|`best_d_one_se`	|integer	|The optimal number of features to include according to one-SE rule.|

In [59]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

Since there are so many columns, the feature selection and model fitting we are about to do can be computationally intensive. Therefore, we'll only consider the first 2000 columns. The next cell will load in that feature data into `X` and the labels into `y`.

In [60]:
X = np.load('X.npy', allow_pickle=True)[:,:2000]
y = np.load('y.npy', allow_pickle=True)

Then, you will set aside 25% of the data for evaluating the final model at the end.  Save the result in `Xtr`, `ytr`, `Xts`, and `yts`. 

Use `sklearn`'s `train_test_split` with shuffling, and you will specify the `random_state = 42`so that your results will match the autograders' results.


In [61]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=0.25, random_state=42)

Now, you will use 10-fold cross validation (with `sklearn`'s `KFold`, no additional shuffling since you have already shuffled the data) to evaluate model candidates as follows:

* First, within each fold, compute the *absolute value* of the correlation coefficient between each column of the feature data and the target variable. (You may use `numpy`'s `corrcoef` function.) Save the results in `score_ft`, which has one entry per column per fold.
* Then, iterate over the number of columns to include in the model - the `d` values in `d_list`. In each iteration, you will use the `d` features that had the highest absolute value of correlation coefficient in the model.
* You will train an SVC model with a linear kernel, `C=10`, `random_state = 24`, and all other settings at their default values. You will evaluate the model on the validation data and save the accuracy score in `score_val`, which has one entry per `d` value per fold.

(Note: in many cases we would standardize the data before fitting an SVC, but we won't do that here.)

Write your solution in the `#grade` cell.

In [62]:
d_list = np.arange(1, X.shape[1]+1) 
nd = len(d_list)
nfold = 10
score_ft = np.zeros((nfold, Xtr.shape[1]))
score_val = np.zeros((nfold, nd))

In [63]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
kf = KFold(n_splits=nfold, shuffle=False)
for i, (train_index, val_index) in enumerate(kf.split(Xtr)):
    X_train, X_val = Xtr[train_index], Xtr[val_index]
    y_train, y_val = ytr[train_index], ytr[val_index]
    for j in range(X_train.shape[1]):
        if i < nfold and j < X_train.shape[1]:
            score_ft[i, j] = np.abs(np.corrcoef(X_train[:, j], y_train)[0, 1])
    for k, d in enumerate(d_list):
        if i < nfold and k < nd:
            top_features = np.argsort(score_ft[i])[-d:]
            X_train_selected = X_train[:, top_features]
            X_val_selected = X_val[:, top_features]
            model = SVC(kernel='linear', C=10, random_state=24)
            model.fit(X_train_selected, y_train)
            y_pred = model.predict(X_val_selected)
            score_val[i, k] = accuracy_score(y_val, y_pred)

Use `score_val` to find `best_d`, the optimal number of features to include in the model (best mean validation accuracy). (Compute the value - don't hard-code it.)

In [64]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
best_d = d_list[np.argmax(np.mean(score_val, axis=1))]

Then, find `best_d_one_se`, the optimal number of features to include according to the one-SE rule:

In [65]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
mean_scores = np.mean(score_val, axis=0)
std_scores = np.std(score_val, axis=0)
best_score = np.max(mean_scores)
best_score_se = std_scores[np.argmax(mean_scores)]
one_se_threshold = best_score - best_score_se
best_d_one_se = d_list[np.min(np.where(mean_scores >= one_se_threshold))]